In [1]:
!pip install openai
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
import os
import json
import openai
from openai import OpenAI
import pandas as pd
import csv
import re
from IPython import get_ipython
from IPython.utils.capture import capture_output
from google.colab import files

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CS 159 Project

Mounted at /content/drive
/content/drive/MyDrive/CS 159 Project


**Starter Code**

In [4]:
YOUR_KEY = "sk-proj-XyubQbMyD0VR84Wd70ygT3BlbkFJsTVSPxQnR4iURk3cdp0v"
client = OpenAI(api_key = YOUR_KEY)

In [5]:
def get_openai_response(prompt):
    response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a highly intelligent AI trained to solve reasoning problems and learn iteratively from feedback."},
                {"role": "user", "content": prompt}
            ]
        )
    return response.choices[0].message.content

**Preprocessing - New Dataset**

In [6]:
from datasets import load_dataset
multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20220616")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3177 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/454 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/908 [00:00<?, ? examples/s]

In [7]:
train = multi_lexsum["train"]
df = pd.DataFrame(train)
summary_proof_concept = df.iloc[0]['summary/long']
print(type(summary_proof_concept))

<class 'str'>


In [8]:
def generate_fact_sheet(summary):
    prompt = f"""
    Given the following legal case summary, create a detailed fact sheet covering the following categories:
    1. Case Information
    2. Parties Involved
    3. Legal Basis
    4. Case Background
    5. Court Proceedings
    6. Settlement and Agreements
    7. Outcome and Impact
    8. Miscellaneous

    Summary: {summary}

    Please provide the fact sheet in a structured format. Output type should
    be a JSON object with the following keys: case_info, parties, legal_basis,
    case_background, court_proceedings, settlement_and_agreements, outcome_and_impact,
    miscellaneous.
    Do not have sub categories in the values of the JSON object.
    Make the values into string types.

    """
    response = get_openai_response(prompt)
    print(response)
    return response

fact_sheet_json_str = generate_fact_sheet(summary_proof_concept)

{
    "case_info": "Case filed by EEOC against House of Philadelphia, Inc. on Sep 15, 2005",
    "parties": "1. Equal Employment Opportunity Commission (EEOC) 2. House of Philadelphia, Inc. 3. Employee (identity not provided)",
    "legal_basis": "Title VII of the Civil Rights Act of 1964 for unlawful discrimination based on sex",
    "case_background": "Employee allegedly fired because she was pregnant, leading to the suit filed by EEOC Seeking monetary and injunctive relief for economic damage, emotional harm compensation, and punitive damages",
    "court_proceedings": "Employee filed motion to intervene and sought relief under Title VII and state law Employee sought reinstatement Parties reached a settlement agreement entered as a consent decree by Judge Kristi K. DuBose on Jan 10, 2009",
    "settlement_and_agreements": "Employee received $8,000 House of Philadelphia, Inc. required to implement anti-discrimination policies, provide training, investigate complaints and report resul

In [9]:
data = json.loads(fact_sheet_json_str)
print(data)
with open("fact_sheet_prelim.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['Category', 'Details'])
    # Write the data
    for key, value in data.items():
      print(key)
      print(value)
      writer.writerow([key, value])

{'case_info': 'Case filed by EEOC against House of Philadelphia, Inc. on Sep 15, 2005', 'parties': '1. Equal Employment Opportunity Commission (EEOC) 2. House of Philadelphia, Inc. 3. Employee (identity not provided)', 'legal_basis': 'Title VII of the Civil Rights Act of 1964 for unlawful discrimination based on sex', 'case_background': 'Employee allegedly fired because she was pregnant, leading to the suit filed by EEOC Seeking monetary and injunctive relief for economic damage, emotional harm compensation, and punitive damages', 'court_proceedings': 'Employee filed motion to intervene and sought relief under Title VII and state law Employee sought reinstatement Parties reached a settlement agreement entered as a consent decree by Judge Kristi K. DuBose on Jan 10, 2009', 'settlement_and_agreements': 'Employee received $8,000 House of Philadelphia, Inc. required to implement anti-discrimination policies, provide training, investigate complaints and report results to EEOC Decree lasted 